# INSTRUCTIONS

1. In the toolbar above, click the "▶ Run" button <strong>THREE TIMES</strong>.
2. Use the sliders to set the initial team censuses (<strong>for simplicity, only Riven 1-5 and VICP are included</strong>), then select the assignment mode. Then click "Start Simulator" and wait a few seconds.
    - A table will appear. If it's too big for your screen, it may help to zoom out your browser to 75% or so.
    - In the table, each cell is a patient. The number in the cell is the patient's length of stay.
5. To discharge a patient, click the corresponding cell.
5. To admit a patient, click "New Patient."
6. To advance to the next day, click "Next Day."

## CELL COLOR KEY
- <span style="color:blue">Blue</span>: Team census cap
- <span style="color:green">Green</span>: New patient
- <span style="color:red">Red</span>: Most recent new patient

<br/>

*Note: "Cumulative LOS" is the average LOS of all discharged and currently admitted patients.*

In [1]:
#https://stackoverflow.com/questions/27934885/how-to-hide-code-from-cells-in-ipython-notebook-visualized-with-nbviewer

import ipywidgets as widgets
from IPython.display import display, HTML

javascript_functions = {False: "hide()", True: "show()"}
button_descriptions  = {False: "Show code", True: "Hide code"}


def toggle_code(state):

    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_functions[state],)
    output        = output_string.format(*output_args)

    display(HTML(output))


def button_action(value):

    state = value.new

    toggle_code(state)

    value.owner.description = button_descriptions[state]


state = False
toggle_code(state)

toggle_button = widgets.ToggleButton(state, description = button_descriptions[state])
toggle_button.observe(button_action, "value")

In [2]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from ipywidgets import HBox, Layout
from IPython.display import display, clear_output, HTML
import random
import re
import time

# Global variable to track the position of the last new patient
last_new_patient_pos = None

# Global dictionary to track the count of all discharged patients for each team
discharged_patients_count = {}

# Global dictionary to track the discharged LOS of all discharged patients for each team (deprecated)
discharged_los_total = {}

# Global dictionary to track the cumulative count of all (admitted and discharged) patients for each team
cumulative_patients_count = {}

# Global dictionary to track the cumulative LOS of all patients for each team
cumulative_los_total = {}

# Global variable for assignment mode
mode = None

# Global sliders
Riven_1_slider = widgets.IntSlider(value=0, min=0, max=20, step=1, description='Riven 1 [20]:')
Riven_2_slider = widgets.IntSlider(value=0, min=0, max=15, step=1, description='Riven 2 [15]:')
Riven_3_slider = widgets.IntSlider(value=0, min=0, max=15, step=1, description='Riven 3 [15]:')
Riven_4_slider = widgets.IntSlider(value=0, min=0, max=15, step=1, description='Riven 4 [15]:')
Riven_5_slider = widgets.IntSlider(value=0, min=0, max=15, step=1, description='Riven 5 [15]:')
VICP_slider = widgets.IntSlider(value=0, min=0, max=20, step=1, description='VICP [20]:')

def create_sliders():
    global mode

    # Display instructions
    header_label = widgets.HTML('<strong>Set initial team censuses:</strong>')
    display(header_label)

    # Display the sliders
    display(Riven_1_slider, Riven_2_slider, Riven_3_slider, Riven_4_slider, Riven_5_slider, VICP_slider)

    # Create Radio Buttons for Mode Selection
    mode_selection = widgets.RadioButtons(
        options=[('Current', 'current'), ('LOSAC', 'losac')],
        description='Select Mode:',
        disabled=False,
        index=None  # No selection by default
    )

    # Display the radio buttons
    display(mode_selection)

    # Initialize the "Start Simulator" button but disable it initially
    start_simulator_button = widgets.Button(description="Start Simulator", disabled=True)
    start_simulator_button.on_click(on_start_simulator_clicked)

    # Function to update mode based on selection
    def on_mode_change(change):
        global mode
        mode = change['new']
        start_simulator_button.disabled = False  # Enable the button

    # Observe changes in radio button selection
    mode_selection.observe(on_mode_change, names='value')

    display(start_simulator_button)

# Function to create initial censuses based on user input from sliders
def create_data_array():
    data = []
    mean = 0
    std_dev = 6 # have set this based on trial and error

    for slider in [Riven_1_slider, Riven_2_slider, Riven_3_slider, Riven_4_slider, Riven_5_slider, VICP_slider]:
        team_data = []
        if slider.value > 0:
            for _ in range(slider.value):
                # Sample from the normal distribution, take the absolute value, and round up so that minimum value is 1
                value = np.ceil(abs(np.random.normal(mean, std_dev)))
                team_data.append(int(value))
        data.append(team_data)

    return data

# Function to be called when "Start Simulator" button is clicked
def on_start_simulator_clicked(b):
    global df, initial_losac_scores, last_new_patient_pos, discharged_patients_count, discharged_los_total, cumulative_patients_count, cumulative_los_total

    # Reset variables
    last_new_patient_pos = None

    # Create the data array from sliders
    data_array = create_data_array()
    team_names = ['Riven 1 [20]', 'Riven 2 [15]', 'Riven 3 [15]', 'Riven 4 [15]', 'Riven 5 [15]', 'VICP [20]']

    # Find the maximum length among all teams
    max_length = max(len(team_data) for team_data in data_array)

    # Pad each team's data to ensure equal length and create the DataFrame
    padded_data = {team: team_data + [None] * (max_length - len(team_data)) for team, team_data in zip(team_names, data_array)}
    df = pd.DataFrame(padded_data)

    # Initialize dataframe to have the maximum census cap as the number of rows
    max_census_cap = max([extract_census_cap(team) for team in df.columns])
    df = df.reindex(range(max_census_cap))

    # Sort each team's list in ascending order of LOS
    df = df.apply(lambda x: pd.to_numeric(x, errors='coerce').sort_values().reset_index(drop=True))

    # Initialize cumulative LOS and patient count dictionaries
    cumulative_los_total = {}
    cumulative_patients_count = {}
    for team in df.columns:
        team_data = df[team].dropna()
        cumulative_los_total[team] = team_data.sum()
        cumulative_patients_count[team] = len(team_data)

    # Initialize the dictionaries with all team names
    discharged_patients_count = {team: 0 for team in df.columns}
    discharged_los_total = {team: 0 for team in df.columns}

    # Calculate initial LOSAC scores
    initial_losac_scores = calculate_losac(df)

    # Call function to draw the table
    clear_output(wait=True)
    draw_table(calculate_losac(df))

# Function to extract the census cap from the column name
def extract_census_cap(column_name):
    match = re.search(r'\[(\d+)\]', column_name)
    if match:
        return int(match.group(1))
    else:
        return None

# Function to calculate LOSAC scores if mode=current
def calculate_losac_current(dataframe):
    losac_scores = {}
    for team in dataframe.columns:
        census_cap = extract_census_cap(team)
        if census_cap is not None:
            team_census = dataframe[team].dropna()
            if team_census.count() < census_cap:
                losac_scores[team] = team_census.count() / census_cap
            else:
                losac_scores[team] = float('inf')
    return losac_scores

# Function to calculate LOSAC scores if mode=losac
def calculate_losac_losac(dataframe):
    losac_scores = {}

    # Calculate the minimum census cap among all teams
    min_census_cap = min([extract_census_cap(team) for team in dataframe.columns])

    for team in dataframe.columns:
        census_cap = extract_census_cap(team)
        if census_cap is not None:
            team_census = dataframe[team].dropna()
            if team_census.count() < census_cap:
                median_los = team_census.median() if team_census.count() > 0 else 1
                losac_scores[team] = (team_census.count() / census_cap) / median_los
            else:
                losac_scores[team] = float('inf')
    return losac_scores

# Dispatcher function for LOSAC calculation
def calculate_losac(dataframe):
    if mode == 'current':
        return calculate_losac_current(dataframe)
    elif mode == 'losac':
        return calculate_losac_losac(dataframe)

# Function to handle discharging patients
def discharge_patient(selected_cell):
    global df, discharged_patients_count, discharged_los_total, cumulative_patients_count, cumulative_los_total

    # Discharge the selected patient
    team, patient = selected_cell.split(',')
    los = df.at[int(patient), team]
    df.at[int(patient), team] = pd.NA

    # Update cumulatived patients count and LOS total
    discharged_patients_count[team] = discharged_patients_count.get(team, 0) + 1
    discharged_los_total[team] = discharged_los_total.get(team, 0) + (los if los is not None else 0)

    # Diagnostic print statement
    #print(f"After discharge: {team} - Cumulative LOS: {cumulative_los_total[team]}, Cumulative count: {cumulative_patients_count[team]}, D/C Count: {discharged_patients_count[team]}")

    # Sort and redraw the table
    df = df.apply(lambda x: pd.to_numeric(x, errors='coerce').sort_values().reset_index(drop=True))
    clear_output(wait=True)
    draw_table(calculate_losac(df))

# Function to admit a new patient if mode=current
def admit_new_patient_current(_):
    global df, last_new_patient_pos, cumulative_los_total, cumulative_patients_count

    # Calculate LOSAC scores with current census
    losac_scores = calculate_losac(df)

    # Filter out teams based on census cap and number of new patients
    eligible_teams = {team: losac for team, losac in losac_scores.items()
                      if df[team].count() < extract_census_cap(team) and
                         ((extract_census_cap(team) == 20 and (df[team] == 1).sum() <= 7) or
                          (extract_census_cap(team) == 15 and (df[team] == 1).sum() <= 3))}

    if eligible_teams:
        # Find the minimum LOSAC score
        min_losac = min(eligible_teams.values())
        tied_teams = [team for team, losac in eligible_teams.items() if losac == min_losac]

        # Handle ties in minimum LOSAC
        if len(tied_teams) > 1:
            # Count the number of patients with LOS of 1 for each tied team
            los_of_1_counts = {team: (df[team] == 1).sum() for team in tied_teams}
            min_los_of_1 = min(los_of_1_counts.values())
            teams_with_min_los_of_1 = [team for team, count in los_of_1_counts.items() if count == min_los_of_1]

            if len(teams_with_min_los_of_1) > 1:
                # Randomly assign if there's still a tie
                selected_team = random.choice(teams_with_min_los_of_1)
            else:
                selected_team = teams_with_min_los_of_1[0]
        else:
            selected_team = tied_teams[0]

        # Insert new patient for the selected team
        df[selected_team] = df[selected_team].shift(1)
        df.at[0, selected_team] = 1
        last_new_patient_pos = (0, selected_team)
        df[selected_team] = pd.to_numeric(df[selected_team], errors='coerce').sort_values().reset_index(drop=True)

        # Update cumulative LOS total and patient count for the selected team
        cumulative_patients_count[selected_team] = cumulative_patients_count.get(selected_team, 0) + 1
        cumulative_los_total[selected_team] = cumulative_los_total.get(selected_team, 0) + 1

    else:
        clear_output(wait=True)
        display(HTML("<div style='color: red; text-align: center;'>All teams have reached their census caps. First discharge a patient.</div>"))
        time.sleep(2)  # Wait for 2 seconds

    # Redraw the table with the updated LOSAC
    clear_output(wait=True)
    draw_table(calculate_losac(df))

# Function to admit a new patient if mode=losac
def admit_new_patient_losac(_):
    global df, last_new_patient_pos, cumulative_los_total, cumulative_patients_count

    # Calculate LOSAC scores with current census
    losac_scores = calculate_losac(df)

    # Filter out teams based on census cap and number of new patients
    eligible_teams = {team: losac for team, losac in losac_scores.items()
                      if df[team].count() < extract_census_cap(team) and
                         ((extract_census_cap(team) == 20 and (df[team] == 1).sum() <= 7) or
                          (extract_census_cap(team) == 15 and (df[team] == 1).sum() <= 3))}

    if eligible_teams:
        # Find the minimum LOSAC score
        min_losac = min(eligible_teams.values())
        tied_teams = [team for team, losac in eligible_teams.items() if losac == min_losac]

        if len(tied_teams) > 1:
            # Determine if all tied teams are at zero census
            all_zero_census = all(df[team].count() == 0 for team in tied_teams)

            # Calculate the % full census for each tied team
            percent_full = {team: df[team].count() / extract_census_cap(team) for team in tied_teams}
            min_percent_full = min(percent_full.values())
            teams_with_min_percent_full = [team for team, percent in percent_full.items() if percent == min_percent_full]

            # If not all teams are at zero census, calculate mean LOS for tie-breaking
            if len(teams_with_min_percent_full) > 1:
                los_means = {team: df[team].mean() for team in teams_with_min_percent_full}
                max_los_mean = max(los_means.values())

                # Account for the fact that mean LOS can't be calculated if all zero census:
                if all_zero_census:
                  teams_with_max_los_mean = teams_with_min_percent_full
                else:
                  teams_with_max_los_mean = [team for team, los in los_means.items() if los == max_los_mean]

                # Randomly select a team if there's still a tie
                if len(teams_with_max_los_mean) > 1:
                    selected_team = random.choice(teams_with_max_los_mean)
                else:
                    selected_team = teams_with_max_los_mean[0]
            else:
                selected_team = teams_with_min_percent_full[0]
        else:
            selected_team = tied_teams[0]

        # Insert new patient for the selected team
        df[selected_team] = df[selected_team].shift(1)
        df.at[0, selected_team] = 1
        last_new_patient_pos = (0, selected_team)
        df[selected_team] = pd.to_numeric(df[selected_team], errors='coerce').sort_values().reset_index(drop=True)

        # Update cumulative LOS total and patient count for the selected team
        cumulative_patients_count[selected_team] = cumulative_patients_count.get(selected_team, 0) + 1
        cumulative_los_total[selected_team] = cumulative_los_total.get(selected_team, 0) + 1
    else:
        clear_output(wait=True)
        display(HTML("<div style='color: red; text-align: center;'>All teams have reached their census caps. First discharge a patient.</div>"))
        time.sleep(2)  # Wait for 2 seconds

    # Redraw the table with the updated LOSAC
    clear_output(wait=True)
    draw_table(calculate_losac(df))

# Dispatcher function for admitting a new patient
def admit_new_patient(_):
    if mode == 'current':
        admit_new_patient_current(_)
    elif mode == 'losac':
        admit_new_patient_losac(_)

# Function to increment the LOS for all patients (start new day)
def increment_day(_):
    global df

    # Increment LOS by 1 for all patients
    df = df.applymap(lambda x: x + 1 if pd.notna(x) else x)

    # Update cumulative LOS total for each team
    for team in df.columns:
        team_data = df[team].dropna()
        cumulative_los_total[team] = cumulative_los_total.get(team, 0) + len(team_data)

    # Sort each team's list in ascending order of LOS
    df = df.apply(lambda x: pd.to_numeric(x, errors='coerce').sort_values().reset_index(drop=True))

    # Redraw the table with updated LOSAC
    clear_output(wait=True)
    draw_table(calculate_losac(df))

# Make sure to update the create_styled_button function with the new parameters
def create_styled_button(description, pos, is_new_patient, is_green, is_blue):
    button = widgets.Button(description=description, layout=widgets.Layout(width='auto'))
    if is_new_patient:
        button.style.button_color = 'lightcoral'  # Light red color for new patient
    elif is_green:
        button.style.button_color = 'lightgreen'  # Light green for patients with LOS of 1
    elif is_blue:
        button.style.button_color = 'lightblue'   # Light blue for patients below census cap
    return button

def draw_table(losac_scores):
    global df, table_grid

    if 'table_grid' not in globals():
        table_grid = widgets.GridspecLayout(df.shape[0] + 4, df.shape[1] + 1)
        initialize_table_grid(table_grid, losac_scores)
    else:
        update_table_grid(table_grid, losac_scores)

    display(table_grid)
    initialize_control_buttons()

def initialize_control_buttons():
    # Define button style and layout
    button_style = {'font_weight': 'bold', 'font_size': '14px'}
    button_layout = Layout(margin='0 25px 0 25px')  # 25px margins on left and right

    # Create buttons
    new_patient_button = widgets.Button(description="New Patient", style=button_style, layout=button_layout, button_style='success')
    new_patient_button.on_click(admit_new_patient)

    next_day_button = widgets.Button(description="Next Day", style=button_style, layout=button_layout, button_style='info')
    next_day_button.on_click(increment_day)

    start_over_button = widgets.Button(description="Start Over", style=button_style, layout=button_layout, button_style='danger')
    start_over_button.on_click(start_over)

    buttons_layout = HBox([new_patient_button, next_day_button, start_over_button],
                           layout=Layout(justify_content='center'))
    display(buttons_layout)

def initialize_table_grid(table, losac_scores):
    global df, last_new_patient_pos, mode

    # Add right-aligned labels to the left of the first column header
    table[0, 0] = widgets.HTML(value="<div style='text-align: right;'>Team [Cap]:</div>")
    table[1, 0] = widgets.HTML(value=f"<div style='text-align: right;'>{'LOSAC / % Full:' if mode == 'current' else 'LOSAC / % Full:'}</div>")
    table[2, 0] = widgets.HTML(value="<div style='text-align: right;'>Discharged Count:</div>")
    table[3, 0] = widgets.HTML(value="<div style='text-align: right;'>Cumulative LOS:</div>")

    # Add headers and initial values for teams
    for j, team in enumerate(df.columns):
        losac_score = losac_scores[team]
        table[0, j + 1] = widgets.HTML(value=f"<center>{team}</center>")
        table[1, j + 1] = widgets.HTML(value=f"<center>{losac_score:.3f}</center>")
        discharged_count = discharged_patients_count.get(team, 0)
        table[2, j + 1] = widgets.HTML(value=f"<center>{discharged_count}</center>")
        cumulative_los = cumulative_los_total.get(team, 0) / cumulative_patients_count.get(team, 1) if cumulative_patients_count.get(team, 1) > 0 else 0
        table[3, j + 1] = widgets.HTML(value=f"<center>{cumulative_los:.2f}</center>")

    # Add row labels and patient buttons
    for i in range(df.shape[0]):
        table[i + 4, 0] = widgets.HTML(value=f"<div style='text-align: right;'>{i + 1}</div>")
        for j in range(df.shape[1]):
            cell_value = df.iloc[i, j]
            if pd.notna(cell_value):
                try:
                    cell_value_str = str(int(cell_value))
                except ValueError:
                    cell_value_str = str(cell_value)
            else:
                cell_value_str = ' '

            is_new_patient = ((i, df.columns[j]) == last_new_patient_pos and cell_value == 1)
            is_green = cell_value == 1 and not is_new_patient
            team_census_cap = extract_census_cap(df.columns[j])
            is_blue = i < team_census_cap
            button = create_styled_button(cell_value_str, pos=(i, j),
                                          is_new_patient=is_new_patient, is_green=is_green, is_blue=is_blue)
            table[i + 4, j + 1] = button
            button.on_click(lambda btn, x=i, y=j: discharge_patient(f"{df.columns[y]},{x}"))

def update_table_grid(table, losac_scores):
    global df

    # Update LOSAC scores and counts
    for j, team in enumerate(df.columns):
        losac_score = losac_scores[team]
        table[1, j + 1].value = f"<center>{losac_score:.3f}</center>"
        discharged_count = discharged_patients_count.get(team, 0)
        table[2, j + 1].value = f"<center>{discharged_count}</center>"
        cumulative_los = cumulative_los_total.get(team, 0) / cumulative_patients_count.get(team, 1) if cumulative_patients_count.get(team, 1) > 0 else 0
        table[3, j + 1].value = f"<center>{cumulative_los:.2f}</center>"

        # Update patient buttons
        for i in range(df.shape[0]):
            cell_value = df.iloc[i, j]
            if pd.notna(cell_value):
                try:
                    cell_value_str = str(int(cell_value))
                except ValueError:
                    cell_value_str = str(cell_value)
            else:
                cell_value_str = ' '

            button = table[i + 4, j + 1]
            update_button(button, cell_value_str, i, j, team)

def update_button(button, value, row, column, team_name):
    global last_new_patient_pos

    # Update the button's description (label)
    button.description = str(value) if value != '' else ' '

    # Update button styles based on the value (e.g., color coding)
    is_new_patient = ((row, team_name) == last_new_patient_pos and value == '1')
    is_green = value == '1' and not is_new_patient
    team_census_cap = extract_census_cap(team_name)
    is_blue = row < team_census_cap
    if is_new_patient:
        button.style.button_color = 'lightcoral'
    elif is_green:
        button.style.button_color = 'lightgreen'
    elif is_blue:
        button.style.button_color = 'lightblue'
    else:
        button.style.button_color = None  # Reset to default color

def reset_table(_):
    global df, initial_losac_scores, last_new_patient_pos, discharged_patients_count

    # Reset variables
    last_new_patient_pos = None
    cumulative_los_total = {}
    cumulative_patients_count = {}

    # Call the function as if the "Start Simulator" button was clicked
    on_start_simulator_clicked(None)

def start_over(_):
    # Reset variables
    cumulative_los_total = {}
    cumulative_patients_count = {}

    # Clear the output
    clear_output(wait=True)
    create_sliders()

# Draw the initial table
create_sliders()

GridspecLayout(children=(HTML(value="<div style='text-align: right;'>Team [Cap]:</div>", layout=Layout(grid_ar…